In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import json
import urllib.request as urllib2
import sys
import time

In [ ]:
# adding path to geckodriver to the OS environment variable
# assuming that it is stored at the same path as this script
os.environ["PATH"] += os.pathsep + os.getcwd()
download_path = "dataset/"

def main(searchtext, n):
    searchtext =  searchtext #search query
    num_requested = n # number of images to download
    number_of_scrolls = int(num_requested / 400 + 1)
    # number_of_scrolls * 400 images will be opened in the browser

    if not os.path.exists(download_path + searchtext.replace(" ", "_")):
        os.makedirs(download_path + searchtext.replace(" ", "_"))

    url = "https://www.google.co.in/search?q="+searchtext+"&source=lnms&tbm=isch"
    driver = webdriver.Chrome()
    driver.get(url)
    
    headers = {}
    #headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome 74.0.3729.169 Safari/537.36"
    headers['User-Agent'] = "Chrome 74.0.3729.169"
    extensions = {"jpg", "jpeg"}
    img_count = 0
    downloaded_img_count = 0
    try:
        driver.find_element_by_xpath("//input[@value='Show more results']").click()
    except Exception as e:
        print ("Less images found:", e)
        #break
        
    for _ in range(number_of_scrolls):
        for __ in range(10):
            # multiple scrolls needed to show all 400 images
            driver.execute_script("window.scrollBy(0, 1000000)")
            time.sleep(0.2)
        # to load next 400 images
        time.sleep(0.5)

    # imges = driver.find_elements_by_xpath('//div[@class="rg_meta"]') # not working anymore
    imges = driver.find_elements_by_xpath('//div[contains(@class,"rg_meta")]')
    print ("Total images:", len(imges), "\n")
    for img in imges:
        img_count += 1
        img_url = json.loads(img.get_attribute('innerHTML'))["ou"]
        if 'oldnavy' in str('{}'.format(img_url)):
            print('olnavy in url ..')
            pass
        elif 'gap' in str('{}'.format(img_url)):
            print('gap in url ..')
            pass
        elif 'express' in str('{}'.format(img_url)):
            print('express in url ..')
            pass
        elif 'childrensplace' in str('{}'.format(img_url)):
            print('express in url ..')
            pass
        else:
            img_type = json.loads(img.get_attribute('innerHTML'))["ity"]
            print ("Downloading image", img_count, ": ", img_url)
            try:
                if img_type not in extensions:
                    img_type = "jpg"
                req = urllib2.Request(img_url, headers=headers)
                raw_img = urllib2.urlopen(req).read()
                f = open(download_path+searchtext.replace(" ", "_")+"/"+str(downloaded_img_count)+"."+img_type, "wb")
                f.write(raw_img)
                f.close
                downloaded_img_count += 1
            except Exception as e:
                print ("Download failed:", e)
            finally:
                print ()
            if downloaded_img_count >= num_requested:
                break

    print ("Total downloaded: ", downloaded_img_count, "/", img_count)
    driver.quit()

if __name__ == "__main__":
    main('bermuda shorts', 2000)

In [50]:
'oldnavy' in str('https://oldnavy.gap.com/webcontent/0016/430/120/cn16430120.jpg')

True